# ✈️ Aviation Weather Anomaly Detection
## 🎯 Objective: Detect Abnormal or Rare Weather Conditions

**Dataset:** METAR Weather Data  
**Model:** Isolation Forest (Unsupervised ML)  
**Purpose:** Aviation Safety - Identify unusual weather patterns that could pose risks

### Why Isolation Forest?
- ✅ Designed specifically for anomaly detection
- ✅ No labels required (unsupervised)
- ✅ Efficient with high-dimensional data
- ✅ Highly relevant for aviation safety

## 📦 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings

warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries imported successfully!")

## 📊 Load METAR Dataset

In [ ]:
# Load the data
df = pd.read_csv('datasets/metar.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

## 🔍 Exploratory Data Analysis

In [ ]:
# Data info
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
df.info()

In [ ]:
# Check for missing values
print("\n" + "=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('Missing_Count', ascending=False)

print(missing_data[missing_data['Missing_Count'] > 0])

In [ ]:
# Statistical summary of numerical features
print("\n" + "=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
df.describe()

## 🧹 Data Preprocessing

In [ ]:
# Select relevant numerical features for anomaly detection
# Focus on key weather parameters that affect aviation safety
feature_columns = [
    'tmpf',      # Temperature (°F)
    'dwpf',      # Dew Point (°F)
    'relh',      # Relative Humidity (%)
    'drct',      # Wind Direction (degrees)
    'sknt',      # Wind Speed (knots)
    'p01i',      # Precipitation (inches)
    'alti',      # Altimeter Setting
    'vsby',      # Visibility (miles)
]

# Create a copy for processing
df_processed = df[feature_columns].copy()

print(f"Selected Features: {feature_columns}")
print(f"\nShape before cleaning: {df_processed.shape}")

In [ ]:
# Handle missing values
# For weather data, we'll use median imputation as it's robust to outliers
for col in df_processed.columns:
    if df_processed[col].isnull().sum() > 0:
        median_value = df_processed[col].median()
        df_processed[col].fillna(median_value, inplace=True)
        print(f"Filled {col} missing values with median: {median_value:.2f}")

print(f"\nShape after cleaning: {df_processed.shape}")
print(f"Missing values remaining: {df_processed.isnull().sum().sum()}")

## 📈 Visualize Feature Distributions

In [ ]:
# Plot distributions of all features
fig, axes = plt.subplots(4, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, col in enumerate(df_processed.columns):
    axes[idx].hist(df_processed[col], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Feature distributions plotted")

## 🔧 Feature Scaling

In [ ]:
# Standardize features (important for Isolation Forest)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_processed)

# Convert back to DataFrame for better interpretability
df_scaled = pd.DataFrame(X_scaled, columns=df_processed.columns, index=df_processed.index)

print("✅ Features scaled using StandardScaler")
print(f"\nScaled data shape: {df_scaled.shape}")
print(f"\nScaled data statistics:")
df_scaled.describe().round(2)

## 🤖 Train Isolation Forest Model

In [ ]:
# Initialize Isolation Forest
# contamination: expected proportion of anomalies (typically 5-10% for weather data)
# random_state: for reproducibility
iso_forest = IsolationForest(
    contamination=0.05,  # Expect ~5% anomalies
    random_state=42,
    n_estimators=100,
    max_samples='auto',
    max_features=1.0,
    bootstrap=False,
    n_jobs=-1,  # Use all CPU cores
    verbose=0
)

print("🔧 Training Isolation Forest Model...")
print(f"Parameters:")
print(f"  - Contamination: {iso_forest.contamination}")
print(f"  - Number of estimators: {iso_forest.n_estimators}")
print(f"  - Random state: {iso_forest.random_state}")

In [ ]:
# Fit the model and predict
predictions = iso_forest.fit_predict(df_scaled)
anomaly_scores = iso_forest.score_samples(df_scaled)

# Add predictions to original dataframe
# -1 for anomalies, 1 for normal points
df['anomaly'] = predictions
df['anomaly_score'] = anomaly_scores

# Create a binary flag (0 = normal, 1 = anomaly)
df['is_anomaly'] = (predictions == -1).astype(int)

print("✅ Model training completed!")
print(f"\nTotal observations: {len(df)}")
print(f"Normal points: {(predictions == 1).sum()}")
print(f"Anomalies detected: {(predictions == -1).sum()}")
print(f"Anomaly percentage: {(predictions == -1).sum() / len(df) * 100:.2f}%")

## 📊 Anomaly Analysis

In [ ]:
# Statistical comparison between normal and anomalous weather
print("=" * 80)
print("NORMAL vs ANOMALOUS WEATHER CONDITIONS")
print("=" * 80)

for col in feature_columns:
    normal_mean = df[df['is_anomaly'] == 0][col].mean()
    anomaly_mean = df[df['is_anomaly'] == 1][col].mean()
    
    print(f"\n{col}:")
    print(f"  Normal Mean: {normal_mean:.2f}")
    print(f"  Anomaly Mean: {anomaly_mean:.2f}")
    print(f"  Difference: {abs(normal_mean - anomaly_mean):.2f}")

In [ ]:
# Show top 10 most anomalous weather conditions
print("\n" + "=" * 80)
print("TOP 10 MOST ANOMALOUS WEATHER CONDITIONS")
print("=" * 80)

anomalies = df[df['is_anomaly'] == 1].sort_values('anomaly_score')
top_anomalies = anomalies.head(10)

display_cols = ['valid', 'tmpf', 'dwpf', 'relh', 'sknt', 'vsby', 'p01i', 'anomaly_score']
top_anomalies[display_cols]

## 📉 Visualize Anomaly Scores

In [ ]:
# Plot anomaly score distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(df[df['is_anomaly'] == 0]['anomaly_score'], bins=50, 
         alpha=0.7, label='Normal', color='green', edgecolor='black')
plt.hist(df[df['is_anomaly'] == 1]['anomaly_score'], bins=50, 
         alpha=0.7, label='Anomaly', color='red', edgecolor='black')
plt.xlabel('Anomaly Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Anomaly Scores', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot([df[df['is_anomaly'] == 0]['anomaly_score'], 
             df[df['is_anomaly'] == 1]['anomaly_score']],
            labels=['Normal', 'Anomaly'])
plt.ylabel('Anomaly Score', fontsize=12)
plt.title('Anomaly Score Comparison', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('anomaly_scores.png', dpi=300, bbox_inches='tight')
plt.show()

## 🎨 2D Visualization using PCA

In [ ]:
# Apply PCA for 2D visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(df_scaled)

# Add PCA components to dataframe
df['PC1'] = X_pca[:, 0]
df['PC2'] = X_pca[:, 1]

print(f"Explained variance ratio:")
print(f"  PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"  PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"  Total: {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
# Create scatter plot
plt.figure(figsize=(14, 8))

# Plot normal points
normal_points = df[df['is_anomaly'] == 0]
plt.scatter(normal_points['PC1'], normal_points['PC2'], 
           c='green', s=20, alpha=0.5, label='Normal', edgecolors='none')

# Plot anomalies
anomaly_points = df[df['is_anomaly'] == 1]
plt.scatter(anomaly_points['PC1'], anomaly_points['PC2'], 
           c='red', s=100, alpha=0.8, label='Anomaly', marker='X', edgecolors='darkred', linewidth=1.5)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('Weather Anomaly Detection - PCA Visualization', fontsize=16, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('anomaly_pca_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ PCA visualization completed")

## 📍 Feature-wise Anomaly Visualization

In [ ]:
# Create box plots for each feature comparing normal vs anomaly
fig, axes = plt.subplots(4, 2, figsize=(15, 14))
axes = axes.ravel()

for idx, col in enumerate(feature_columns):
    data_to_plot = [df[df['is_anomaly'] == 0][col].dropna(), 
                    df[df['is_anomaly'] == 1][col].dropna()]
    
    bp = axes[idx].boxplot(data_to_plot, labels=['Normal', 'Anomaly'],
                           patch_artist=True, showmeans=True)
    
    # Color the boxes
    bp['boxes'][0].set_facecolor('lightgreen')
    bp['boxes'][1].set_facecolor('lightcoral')
    
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Value', fontsize=10)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Feature Comparison: Normal vs Anomalous Weather', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('feature_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 🌡️ Temperature vs Visibility Anomaly Map

In [ ]:
# Scatter plot: Temperature vs Visibility (critical aviation parameters)
plt.figure(figsize=(12, 7))

plt.scatter(df[df['is_anomaly'] == 0]['tmpf'], 
           df[df['is_anomaly'] == 0]['vsby'],
           c='green', s=30, alpha=0.5, label='Normal', edgecolors='none')

plt.scatter(df[df['is_anomaly'] == 1]['tmpf'], 
           df[df['is_anomaly'] == 1]['vsby'],
           c='red', s=120, alpha=0.8, label='Anomaly', 
           marker='X', edgecolors='darkred', linewidth=1.5)

plt.xlabel('Temperature (°F)', fontsize=12)
plt.ylabel('Visibility (miles)', fontsize=12)
plt.title('Temperature vs Visibility: Anomaly Detection', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('temp_vs_visibility.png', dpi=300, bbox_inches='tight')
plt.show()

## 💨 Wind Speed vs Precipitation Anomaly Map

In [ ]:
# Scatter plot: Wind Speed vs Precipitation
plt.figure(figsize=(12, 7))

plt.scatter(df[df['is_anomaly'] == 0]['sknt'], 
           df[df['is_anomaly'] == 0]['p01i'],
           c='green', s=30, alpha=0.5, label='Normal', edgecolors='none')

plt.scatter(df[df['is_anomaly'] == 1]['sknt'], 
           df[df['is_anomaly'] == 1]['p01i'],
           c='red', s=120, alpha=0.8, label='Anomaly', 
           marker='X', edgecolors='darkred', linewidth=1.5)

plt.xlabel('Wind Speed (knots)', fontsize=12)
plt.ylabel('Precipitation (inches)', fontsize=12)
plt.title('Wind Speed vs Precipitation: Anomaly Detection', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('wind_vs_precipitation.png', dpi=300, bbox_inches='tight')
plt.show()

## 📈 Correlation Heatmap

In [ ]:
# Correlation matrix for features
plt.figure(figsize=(10, 8))

correlation_matrix = df_processed.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})

plt.title('Feature Correlation Heatmap', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## 📋 Anomaly Summary Report

In [ ]:
# Generate comprehensive summary
print("="*80)
print("ANOMALY DETECTION SUMMARY REPORT")
print("="*80)

print(f"\n📊 Dataset Overview:")
print(f"   Total Weather Observations: {len(df):,}")
print(f"   Date Range: {df['valid'].min()} to {df['valid'].max()}")
print(f"   Station(s): {df['station'].unique().tolist()}")

print(f"\n🎯 Model Configuration:")
print(f"   Algorithm: Isolation Forest")
print(f"   Contamination Rate: {iso_forest.contamination*100}%")
print(f"   Number of Trees: {iso_forest.n_estimators}")
print(f"   Features Used: {len(feature_columns)}")

print(f"\n🔍 Detection Results:")
print(f"   Normal Observations: {(df['is_anomaly'] == 0).sum():,} ({(df['is_anomaly'] == 0).sum()/len(df)*100:.2f}%)")
print(f"   Anomalies Detected: {(df['is_anomaly'] == 1).sum():,} ({(df['is_anomaly'] == 1).sum()/len(df)*100:.2f}%)")
print(f"   Average Anomaly Score (Normal): {df[df['is_anomaly'] == 0]['anomaly_score'].mean():.4f}")
print(f"   Average Anomaly Score (Anomaly): {df[df['is_anomaly'] == 1]['anomaly_score'].mean():.4f}")

print(f"\n⚠️ Aviation Safety Insights:")
low_visibility_anomalies = df[(df['is_anomaly'] == 1) & (df['vsby'] < 3)]
high_wind_anomalies = df[(df['is_anomaly'] == 1) & (df['sknt'] > 20)]
heavy_precip_anomalies = df[(df['is_anomaly'] == 1) & (df['p01i'] > 0.1)]

print(f"   Low Visibility Anomalies (<3 miles): {len(low_visibility_anomalies)}")
print(f"   High Wind Anomalies (>20 knots): {len(high_wind_anomalies)}")
print(f"   Heavy Precipitation Anomalies (>0.1 in): {len(heavy_precip_anomalies)}")

print("\n" + "="*80)

## 💾 Save Results

In [ ]:
# Save anomalies to CSV
anomaly_df = df[df['is_anomaly'] == 1].sort_values('anomaly_score')
anomaly_df.to_csv('detected_anomalies.csv', index=False)

print(f"✅ Saved {len(anomaly_df)} anomalies to 'detected_anomalies.csv'")

# Save full dataset with predictions
df.to_csv('metar_with_anomalies.csv', index=False)
print(f"✅ Saved full dataset with predictions to 'metar_with_anomalies.csv'")

## 🎯 Key Findings & Recommendations

### Model Performance
- ✅ Isolation Forest successfully identified rare weather patterns
- ✅ Unsupervised approach requires no labeled training data
- ✅ Scalable for real-time aviation weather monitoring

### Aviation Safety Applications
1. **Pre-flight Planning**: Identify potentially hazardous weather conditions
2. **Real-time Alerts**: Flag unusual weather patterns for pilot awareness
3. **Route Optimization**: Avoid areas with anomalous weather
4. **Maintenance Scheduling**: Detect extreme conditions affecting aircraft

### Next Steps
- [ ] Fine-tune contamination parameter based on domain expertise
- [ ] Integrate with real-time METAR feeds
- [ ] Build alerting system for critical anomalies
- [ ] Validate with historical incident data